In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
import random

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sS = pd.read_csv("sampleSubmission.csv")

In [3]:
select_label = [0, 1, 2, 3]
select_habits = [0., 1.]

In [4]:
train['推薦類型'].replace(['肌力',"柔軟度","心肺","肌耐力"],[0,1,2,3] , inplace = True)

In [5]:
train = train[train['推薦類型'].isin(select_label)]
train = train[train['拳擊'].isin(select_habits)]
train = train[train['器材'].isin(select_habits)]
train = train[train['有氧'].isin(select_habits)]
train = train[train['舞蹈'].isin(select_habits)]
train = train[train['高強度間歇'].isin(select_habits)]
train = train[train['瑜伽'].isin(select_habits)]
train = train[train['肌力訓練'].isin(select_habits)]
train = train[train['燃脂'].isin(select_habits)]
train = train[train['減脂瘦身'].isin(select_habits)]
train = train[train['增加肌肉'].isin(select_habits)]
train = train[train['增強體魄'].isin(select_habits)]
train = train[train['舒緩壓力'].isin(select_habits)]
train = train[train['年齡']<70]
train = train[train['BMI']>10]

In [6]:
train['推薦類型'] = train['推薦類型'].astype("int64")
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3425 entries, 5 to 26416
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   性別      3425 non-null   float64
 1   年齡      3425 non-null   float64
 2   BMI     3425 non-null   float64
 3   拳擊      3425 non-null   float64
 4   器材      3425 non-null   float64
 5   有氧      3425 non-null   float64
 6   舞蹈      3425 non-null   float64
 7   高強度間歇   3425 non-null   float64
 8   瑜伽      3425 non-null   float64
 9   肌力訓練    3425 non-null   float64
 10  燃脂      3425 non-null   float64
 11  減脂瘦身    3425 non-null   float64
 12  增加肌肉    3425 non-null   float64
 13  增強體魄    3425 non-null   float64
 14  舒緩壓力    3425 non-null   float64
 15  推薦類型    3425 non-null   int64  
dtypes: float64(15), int64(1)
memory usage: 454.9 KB


In [7]:
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
X = train.drop(["推薦類型"], axis =1)
y = train["推薦類型"]

scaler = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled)

X_s, y_s =  TomekLinks().fit_resample(X_scaled, y)
X_s, y_s = SMOTE(random_state = 0).fit_resample(X_scaled,y)

X_train, X_val, y_train, y_val = train_test_split(
   X_s, y_s, test_size=0.33, random_state=42)

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
SV = SVC(C = 80,gamma =1.7,kernel='poly')
SV.fit(X_train,y_train)
SV.score(X_val,y_val)

In [ ]:
for i in range(1,25):
    SV = SVC(C = 80,gamma =0.1*i)
    SV.fit(X_train,y_train)
    print(SV.score(X_val,y_val))

In [8]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
k_scores = []
for i in range(1,10):
    rnd = RandomForestClassifier(random_state=100,n_estimators=100*i,criterion='entropy')
    scores = cross_val_score(rnd,X_s,y_s,cv=4,scoring='accuracy')
    k_scores.append(scores.mean())
k_scores    

KeyboardInterrupt: 

In [ ]:
k_scores = []
for i in range(1,10):
    xg =  XGBClassifier(random_state=100, n_estimators=100*i,learning_rate=0.15)
    scores = cross_val_score(xg,X_s,y_s,cv=4,scoring='accuracy')
    k_scores.append(scores.mean())
k_scores    

In [ ]:
k_scores = []
for i in range(1,20):
    sv =  SVC(random_state=100,C=10*i ,gamma = 1.7)
    scores = cross_val_score(sv,X_s,y_s,cv=4,scoring='accuracy')
    k_scores.append(scores.mean())
k_scores    

In [9]:
def accuracy_score(y_val,y_pred):
    Sum = 0
    y_val = y_val.tolist()
    y_pred = y_pred.tolist()
    for i in range(len(y_pred)):
        if y_val[i] == y_pred[i]:
            Sum+=1
    score = Sum/len(y_pred)
    
    return score


rnd_clf = RandomForestClassifier(random_state=100,n_estimators=700)
xgb_clf =  XGBClassifier(random_state=100, n_estimators=300,learning_rate=0.15)
svc_clf = SVC(C= 80, gamma = 1.7,kernel='poly')

voting_clf = VotingClassifier(estimators=[('rf', rnd_clf), ('sv', svc_clf)],voting='hard')
scores_1 = cross_val_score(rnd_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores_2 = cross_val_score(xgb_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores_3 = cross_val_score(svc_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores = cross_val_score(voting_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
print("rnd: "+str(scores_1.mean()))
print(" xgb: "+str(scores_2.mean()))
print(" svc: "+str(scores_3.mean()))
print(" vote: "+str(scores.mean()))

rnd: 0.8689883913764511
 xgb: 0.8476368159203981
 svc: 0.8186152570480928
 vote: 0.8291873963515756


In [12]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(test)
test = scaler.transform(test)
test = pd.DataFrame(test)




voting_clf.fit(X_s,y_s)
y_pred = voting_clf.predict(test)
y_pred_pd = pd.DataFrame(data = y_pred, columns = ["label"])
y_pred_pd = y_pred_pd.astype('int64').reset_index()
y_pred_pd
y_pred_pd.to_csv("e94084032.csv", index = False)